resimde belli piksel aralığı seçme roi

In [ ]:
import cv2

img = cv2.imread("klon.jpg")

roi = img[30:200, 200:400]

cv2.imshow("klon", img)
cv2.imshow("ROI", roi)

cv2.waitKey(0)
cv2.destroyAllWindows

resimi pencerede kaydırma

In [ ]:
import cv2
import numpy as np

img = cv2.imread("klon.jpg")

row,col = img.shape[:2]

M= np.float32([[1,0,50],[0,1,100]])

dst = cv2.warpAffine(img,M,(row,col))

cv2.imshow("dst", dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

resmi pencerede döndürme

In [ ]:
import cv2
import numpy as np

img = cv2.imread("klon.jpg")

row,col = img.shape[:2]

M = cv2.getRotationMatrix2D((col/2,row/2),90,1) #satır sütün bölüyoz, döndürme oranı, aynısını yapıyoz

dst = cv2.warpAffine(img,M,(row,col))

cv2.imshow("dst", dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

threshold

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as pyplot

img = cv2.imread("klon.jpg",0)

ret,th1 =cv2.threshold(img,150,200,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,21,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,21,2)
cv2.imshow("th1",th1)
cv2.imshow("th2",th2)
cv2.imshow("th3",th3)

cv2.waitKey(0)
cv2.destroyAllWindows()

köşe bulma

In [1]:
import cv2
import numpy as np

img = cv2.imread("text.png")
img1 = cv2.imread("contour.png")

gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)

corners = cv2.goodFeaturesToTrack(gray, 1000, 0.01, 10) #max köşe sayısı, kalite sayısı, min uzaklık bilgisi
corners= np.int32(corners)

for corner in corners:
    x,y = corner.ravel()
    cv2.circle(img1, (x,y), 3, (0,0,255), -1)
    
cv2.imshow("corner", img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

canny kenar saptama

In [2]:
import cv2

cap = cv2.VideoCapture(0)

while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    edges = cv2.Canny(frame, 100, 200)
    
    cv2.imshow("frame", frame)
    cv2.imshow("edges", edges)
    
    if cv2.waitKey(5) & 0xFF == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()

contours ilk adım

In [3]:
import cv2

img = cv2.imread("contour2.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
contours,ret = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(img, contours, -1, (0,0,255), 5)

cv2.imshow("contour", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

nesneleri izleme

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("dog.mp4")

while(1):
    ret,frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    sensitivy = 15
    lower_white = np.array([0,0,255-sensitivy])
    upper_white = np.array([255,sensitivy,255])
    mask = cv2.inRange(hsv,lower_white,upper_white)
    res = cv2.bitwise_and(frame,frame,mask=mask)
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask", mask)
    cv2.imshow("result",res)
    
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
cv2.destroyAllWindows()

dışbükey çevre çizme

In [4]:
import cv2

img = cv2.imread("star.png")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(gray,127,255,0)

contours,_ = cv2.findContours(thresh,2,1)
cnt = contours[0]

hull = cv2.convexHull(cnt, returnPoints = False)
defects = cv2.convexityDefects(cnt,hull)

for i in range(defects.shape[0]):
    s,e,f,d = defects[i,0]
    start = tuple(cnt[s][0]) #başlama noktası
    end = tuple(cnt[e][0]) #bitiş noktası
    far = tuple(cnt[f][0]) #uzaklık diğeride uzunluk d olan
    cv2.line(img,start,end,[0,255,0],2)
    cv2.circle(img,far,5,[0,255,0],-1)
    
cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()  

map çevre ve sınır

In [5]:
import cv2
import numpy as np

img = cv2.imread("map.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.blur(gray,(3,3))
ret,thresh = cv2.threshold(blur,40,255,cv2.THRESH_BINARY)

contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

hull = []

for i in range(len(contours)):
    hull.append(cv2.convexHull(contours[i],False))

bg = np.zeros((thresh.shape[0],thresh.shape[1],3),np.uint8)

for i in range(len(contours)):
    cv2.drawContours(bg,contours,i,(255,0,0),3,8,hierarchy)
    cv2.drawContours(bg,hull,i,(0,255,0),3,8)
    
cv2.imshow("img",bg)
cv2.waitKey(0)
cv2.destroyAllWindows()

hough line transform resim

In [14]:
import cv2
import numpy as np

img = cv2.imread("h_line.png")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,75,150)

lines = cv2.HoughLinesP(edges,1,np.pi/180,50,maxLineGap=200) # max line gap bulunmayan kenarlarıda kapsayarak iyileştirmeye yarıyor / p li olanı kullanıyoruz cpu

for line in lines:
    x1,y1,x2,y2 = line[0] #linesı yazdırığımızda 4 değişken veriyor bizimde 4 yani
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    
cv2.imshow("img",img)
cv2.imshow("gray",gray)
cv2.imshow("edges",edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

hough line transform video

In [10]:
import cv2
import numpy as np

cap = cv2.VideoCapture("yellowline.mp4")

while True:
    ret,frame = cap.read()
    frame = cv2.resize(frame,(640,480))
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lower_yellow = np.array([18,94,140],np.uint8) #np.uint8 fln aralık tutulan
    upper_yellow = np.array([48,255,255],np.uint8)
    
    mask = cv2.inRange(hsv,lower_yellow,upper_yellow)
    edges = cv2.Canny(mask,75,250)
    
    lines = cv2.HoughLinesP(edges,1,np.pi/180,50,maxLineGap=50)
    
    for line in lines:
        (x1,y1,x2,y2) = line[0] 
        cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),5)
        
    cv2.imshow("cap",frame)
    
    if cv2.waitKey(20) & 0xFF == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()

hough line transform şekil

In [5]:
import cv2
import numpy as np

img1 = cv2.imread("coins.jpg")
img2 = cv2.imread("balls.jpg")

gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

img1_blur = cv2.medianBlur(gray1,5)
img2_blur = cv2.medianBlur(gray2,5)

circles1 = cv2.HoughCircles(img1_blur,cv2.HOUGH_GRADIENT,1,img1.shape[0]/4,param1=200,param2=10,minRadius=15,maxRadius=89)
circles2 = cv2.HoughCircles(img2_blur,cv2.HOUGH_GRADIENT,1,img2.shape[0]/4,param1=200,param2=10,minRadius=15,maxRadius=89) # shapele aralık belirliyoruz önemli

if circles1 is not None:
    circles1 = np.uint16(np.around(circles1))
    for i in circles1[0,:]: #0 dan başlıyor sona kadar i ye atıyor hepsini alıyoz
        cv2.circle(img1,(i[0],i[1]),i[2],(0,255,0),2) # resim,merkez koordinatları,yarıçap

if circles2 is not None:
    circles2 = np.uint16(np.around(circles2))
    for i in circles2[0,:]: #0 dan başlıyor sona kadar i ye atıyor hepsini alıyoz
        cv2.circle(img2,(i[0],i[1]),i[2],(0,255,0),2) # resim,merkez koordinatları,yarıçap

cv2.imshow("img1",img1)
cv2.imshow("img2",img2)

cv2.waitKey(0)
cv2.destroyAllWindows()

çokgen algılama

In [ ]:
import cv2
import numpy as np

img = cv2.imread("polygons.png")
font = cv2.FONT_HERSHEY_PLAIN

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
_,threshold = cv2.threshold(gray,240,255,cv2.THRESH_BINARY) #24,255 SINIRLAR

contours,_ = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    epsilon = 0.01*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    
    cv2.drawContours(img,[approx],0,(0),5)
    
    x = approx.ravel()[0]
    y = approx.ravel()[1]
    
    if len(approx) == 3:
        cv2.putText(img,"triangle",(x,y),font,3,(0))
        
    elif len(approx) == 4:
        cv2.putText(img,"rectangle",(x,y),font,3,(0))
        
    elif len(approx) == 5:
        cv2.putText(img,"pentagon",(x,y),font,3,(0)) #(0) renk parametresi
        
    elif len(approx) == 6:
        cv2.putText(img,"hexagon",(x,y),font,3,(0))
        
    else:
        cv2.putText(img,"ellipse",(x,y),font,3,(0))

cv2.imshow("img",img)

cv2.waitKey(0)
cv2.destroyAllWindows()

gerçek zamanlı şekil algılama

In [1]:
import cv2
import numpy as np

def nothing(x):
    pass

cap =cv2.VideoCapture(0)
cv2.namedWindow("Settings")

cv2.createTrackbar("Lower_Hue","Settings",0,180,nothing)
cv2.createTrackbar("Lower_Saturation","Settings",0,255,nothing)
cv2.createTrackbar("Lower_Value","Settings",0,255,nothing)
cv2.createTrackbar("Upper_Hue","Settings",0,180,nothing)
cv2.createTrackbar("Upper_Saturation","Settings",0,255,nothing)
cv2.createTrackbar("Upper_Value","Settings",0,255,nothing)

font = cv2.FONT_HERSHEY_SIMPLEX

while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lh = cv2.getTrackbarPos("Lower_Hue","Settings")
    ls = cv2.getTrackbarPos("Lower_Saturation","Settings")
    lv = cv2.getTrackbarPos("Lower_Value","Settings")
    uh = cv2.getTrackbarPos("Upper_Hue","Settings")
    us = cv2.getTrackbarPos("Upper_Saturation","Settings")
    uv = cv2.getTrackbarPos("Upper_Value","Settings")
    
    lower_color = np.array([lh,ls,lv])
    upper_color = np.array([uh,us,uv])
    
    mask =cv2.inRange(hsv,lower_color,upper_color)
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.erode(mask,kernel)
    
    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        epsilon = 0.01*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
    
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        
        if area > 400:
            cv2.drawContours(frame,[approx],0,(0,0,0),5)
            
            if len(approx) == 3:
                cv2.putText(frame,"triangle",(x,y),font,1,(0,0,0))
                
            elif len(approx) == 4:
                cv2.putText(frame,"rectangle",(x,y),font,1,(0,0,0))

            elif len(approx) > 6:
                cv2.putText(frame,"cricle",(x,y),font,1,(0,0,0))
                
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)

    if cv2.waitKey(3) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

arka plan çıkarma

In [ ]:
import cv2
import numpy as np

cap =cv2.VideoCapture("car.mp4")
_,first_frame =cap.read()
first_frame = cv2.resize(first_frame,(640,480))
first_gray = cv2.cvtColor(first_frame,cv2.COLOR_BGR2GRAY)
first_gray = cv2.GaussianBlur(first_gray,(5,5),0)

while 1:
    _,frame = cap.read()
    frame = cv2.resize(frame,(640,480))
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(5,5),0)
    diff = cv2.absdiff(first_gray,gray)
    _,diff = cv2.threshold(diff,25,255,cv2.THRESH_BINARY)
    
    cv2.imshow("frame",frame)
    cv2.imshow("first",first_frame)
    cv2.imshow("diff",diff)
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

otomatik arka plan çıkarma 

In [ ]:
import cv2
import numpy as np

cap =cv2.VideoCapture("car.mp4")
subtractor =cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=120,detectShadows=True) #history değeri frame sayısı

while 1:
    _,frame = cap.read()
    frame = cv2.resize(frame,(640,480))
    mask = subtractor.apply(frame)
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

hsv ile nesne tespiti

In [2]:
import cv2
import numpy as np

def nothing(x):
    pass

cap =cv2.VideoCapture("hsv.mp4")
cv2.namedWindow("Trackbar")

cv2.createTrackbar("LH","Trackbar",0,179,nothing)
cv2.createTrackbar("LS","Trackbar",0,255,nothing)
cv2.createTrackbar("LV","Trackbar",0,255,nothing)
cv2.createTrackbar("UH","Trackbar",0,179,nothing)
cv2.createTrackbar("US","Trackbar",0,255,nothing)
cv2.createTrackbar("UV","Trackbar",0,255,nothing)


while 1:
    ret,frame = cap.read()
    
    if ret is False:
        continue
    
    frame = cv2.resize(frame,(500,350))
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lh = cv2.getTrackbarPos("LH","Trackbar")
    ls = cv2.getTrackbarPos("LS","Trackbar")
    lv = cv2.getTrackbarPos("LV","Trackbar")
    uh = cv2.getTrackbarPos("UH","Trackbar")
    us = cv2.getTrackbarPos("US","Trackbar")
    uv = cv2.getTrackbarPos("UV","Trackbar")
    
    lower_blue = np.array([lh,ls,lv])
    upper_blue = np.array([uh,us,uv])
    
    mask =cv2.inRange(hsv,lower_blue,upper_blue)
    bitwise = cv2.bitwise_and(frame,frame,mask=mask)
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    cv2.imshow("bitwise",bitwise)

    if cv2.waitKey(20) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

hsv ile nesne tespiti webcam

In [ ]:
import cv2
import numpy as np

def nothing(x):
    pass

cap =cv2.VideoCapture(0)
cv2.namedWindow("Trackbar")

cv2.createTrackbar("LH","Trackbar",0,179,nothing)
cv2.createTrackbar("LS","Trackbar",0,255,nothing)
cv2.createTrackbar("LV","Trackbar",0,255,nothing)
cv2.createTrackbar("UH","Trackbar",0,179,nothing)
cv2.createTrackbar("US","Trackbar",0,255,nothing)
cv2.createTrackbar("UV","Trackbar",0,255,nothing)


while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    frame = cv2.resize(frame,(500,350))
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lh = cv2.getTrackbarPos("LH","Trackbar")
    ls = cv2.getTrackbarPos("LS","Trackbar")
    lv = cv2.getTrackbarPos("LV","Trackbar")
    uh = cv2.getTrackbarPos("UH","Trackbar")
    us = cv2.getTrackbarPos("US","Trackbar")
    uv = cv2.getTrackbarPos("UV","Trackbar")
    
    lower_blue = np.array([lh,ls,lv])
    upper_blue = np.array([uh,us,uv])
    
    mask =cv2.inRange(hsv,lower_blue,upper_blue)
    bitwise = cv2.bitwise_and(frame,frame,mask=mask)
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    cv2.imshow("bitwise",bitwise)

    if cv2.waitKey(20) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

fare ile videoya daire çizme (fare kullanımı)

In [ ]:
import cv2

cap = cv2.VideoCapture("car.mp4")
circles = []

def mouse(event,x,y,flags,params):
    if event == cv2.EVENT_LBUTTONDOWN:
        circles.append((x,y))

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame",mouse)

while 1:
    _,frame = cap.read()
    frame = cv2.resize(frame,(640,480))
    for center in circles:
        cv2.circle(frame,center,20,(255,0,0),-1)
        
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1)
    if key ==27:
        break
    elif key ==ord("h"):
        circles = []
        
cap.release()
cv2.destroyAllWindows()

yüz özelliklerini kullanma

In [ ]:
import cv2
import numpy as np
import math

def findMaxContour(contours):
    max_i = 0
    max_area =0
    for i in range(len(contours)):
        area_face = cv2.contourArea(contours[i])
        
        if max_area<area_face:
            max_area=area_face
            max_i = i
        try:
            c = contours[max_i]            
        except:
            contours = [0]
            c = contours[0]
        return c  


cap  =cv2.VideoCapture(0)


while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    roi = frame[50:250,200:400] # frame[y1:y2,x1:x2]
    cv2.rectangle(frame,(200,50),(400,250),(0,0,255),0)

    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    lower_color = np.array([0,45,79],dtype=np.uint8)
    upper_color = np.array([17,255,255],dtype=np.uint8)

    mask  =cv2.inRange(hsv,lower_color,upper_color)
    
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations=1)
    mask = cv2.medianBlur(mask,15)

    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
      
        c = findMaxContour(contours)
            
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        cv2.circle(roi,extLeft,5,(0,255,0),2)
        cv2.circle(roi,extRight,5,(0,255,0),2)
        cv2.circle(roi,extTop,5,(0,255,0),2)
        cv2.circle(roi,extBot,5,(0,255,0),2)

        cv2.line(roi,extLeft,extTop,(255,0,0),2)
        cv2.line(roi,extTop,extRight,(255,0,0),2)
        cv2.line(roi,extRight,extBot,(255,0,0),2)
        cv2.line(roi,extBot,extLeft,(255,0,0),2)

        a = math.sqrt((extRight[0]-extTop[0])**2+(extRight[1]-extTop[1])**2)
        b = math.sqrt((extBot[0]-extRight[0])**2+(extBot[1]-extRight[1])**2)
        c = math.sqrt((extBot[0]-extTop[0])**2+(extBot[1]-extTop[1])**2)

        try:
            angle_ab= int(math.acos((a**2+b**2-c**2)/(2*b*c))*57)
            cv2.putText(roi,str(angle_ab),(extRight[0]-100+50,extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
        except:
            cv2.putText(roi," ? ",(extRight[0]-100+50,extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
   
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

el özelliklerini kullanma

In [ ]:
import cv2
import numpy as np
import math

def findMaxContour(contours):
    max_i = 0
    max_area =0
    for i in range(len(contours)):
        area_hand = cv2.contourArea(contours[i])
        
        if max_area<area_hand:
            max_area=area_hand
            max_i = i
        try:
            c = contours[max_i]            
        except:
            contours = [0]
            c = contours[0]
        return c  


cap  =cv2.VideoCapture(0)


while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    roi = frame[50:250,200:400] # frame[y1:y2,x1:x2]
    cv2.rectangle(frame,(200,50),(400,250),(0,0,255),0)

    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    lower_color = np.array([0,45,79],dtype=np.uint8)
    upper_color = np.array([17,255,255],dtype=np.uint8)

    mask  =cv2.inRange(hsv,lower_color,upper_color)
    
    kernel = np.ones((3,3),np.uint8)
    mask = cv2.dilate(mask,kernel,iterations=1)
    mask = cv2.medianBlur(mask,15)

    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
      
        c = findMaxContour(contours)
            
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])

        cv2.circle(roi,extLeft,5,(0,255,0),2)
        cv2.circle(roi,extRight,5,(0,255,0),2)
        cv2.circle(roi,extTop,5,(0,255,0),2)

        cv2.line(roi,extLeft,extTop,(255,0,0),2)
        cv2.line(roi,extTop,extRight,(255,0,0),2)
        cv2.line(roi,extRight,extLeft,(255,0,0),2)

        a = math.sqrt((extRight[0]-extTop[0])**2+(extRight[1]-extTop[1])**2)
        c = math.sqrt((extTop[0]-extLeft[0])**2+(extTop[1]-extLeft[1])**2)
        b = math.sqrt((extRight[0]-extLeft[0])**2+(extRight[1]-extLeft[1])**2)
        try:
            angle_ab= int(math.acos((a**2+b**2-c**2)/(2*b*c))*57)
            cv2.putText(roi,str(angle_ab),(extRight[0]-100+50,extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            
            if angle_ab>70:
                cv2.rectangle(frame,(0,0),(100,100),-1)
            else:
                pass
            
        except:
            cv2.putText(roi," ? ",(extRight[0]-100+50,extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
   
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

göz izleme

In [ ]:
import cv2

vid = cv2.VideoCapture("eye.mp4")

while 1:
    ret,frame=vid.read()
    if ret is False:
        break

    roi = frame[80:210,230:450]
    rows,cols,_ = roi.shape
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)

    _,threshold = cv2.threshold(gray,3,255,cv2.THRESH_BINARY_INV)

    contours,_ = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = lambda x: cv2.contourArea(x), reverse=True)

    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        cv2.rectangle(roi,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.line(roi,(x+int(w/2),0),(x+int(w/2),rows),(0,255,0),2)
        cv2.line(roi,(0,y+int(h/2)),(cols,y+int(h/2)),(0,255,0),2)
        break
    
    frame[80:210,230:450] = roi
    
    cv2.imshow("frame",frame)
  
    if cv2.waitKey(80) & 0xFF==ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

resim karşılaştırma

In [3]:
import cv2
import numpy as np

img1 = cv2.imread("aircraft.jpg")
img1 = cv2.resize(img1,(640,550))

img2 = cv2.imread("aircraft2.jpg")
img2 = cv2.resize(img2,(640,550))

img3 = cv2.medianBlur(img1,7)

diff = cv2.subtract(img1,img3) #farkları buluyor
b,g,r = cv2.split(diff)

if cv2.countNonZero(b) ==0 and cv2.countNonZero(g) ==0 and cv2.countNonZero(r) ==0:
    print("completely equal")
else:
    print("NOT completely equal")
    
cv2.imshow("Difference",diff)
cv2.waitKey(0)
cv2.destroyAllWindows()

NOT completely equal


çözünürlük ayarlama/pencere boyutu farklaşma

In [ ]:
import cv2

windowName = "Live Video"
cv2.namedWindow(windowName)

cap = cv2.VideoCapture(0)

print("Width : "+str(cap.get(3))) # en oranı 3 boy 4
print("Height : "+str(cap.get(4)))

cap.set(3,1280)
cap.set(4,720)

print("Width* : "+str(cap.get(3)))
print("Height* : "+str(cap.get(4)))

while True:
    _,frame = cap.read()
    frame = cv2.flip(frame,1)

    cv2.imshow(windowName,frame)

    if cv2.waitKey(80) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

blurlu resim bulma

In [ ]:
import cv2

img = cv2.imread("starwars.jpg")
blurry_img = cv2.medianBlur(img,9)

laplacian = cv2.Laplacian(blurry_img,cv2.CV_64F).var() #blur bulma değer veriyoruz aşaüıda ifte
print(laplacian)

if laplacian < 500:
    print("blurry image")
    
cv2.imshow("img",img)
cv2.imshow("blurry_img",blurry_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

şablon eşleştirme

In [4]:
import cv2
import numpy as np

image_path ="starwars.jpg"
template_path ="starwars2.jpg"

img = cv2.imread(image_path)
gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

template = cv2.imread(template_path,cv2.IMREAD_GRAYSCALE) # CV2. siyah beyaza çevirmeye yarar
w,h = template.shape[::-1] #-1 yükseklik ve boy yerini değişiyoruz -1 yazmasaydık h,w yazmalıydık
# print(template.shape)

result = cv2.matchTemplate(gray_img,template,cv2.TM_CCOEFF_NORMED) #ilk şablonun yerleşeceği resim,şablon
location = np.where(result >= 0.95)

for point in zip(*location[::-1]): # zip(* anlamlı koordinatlar elde etmek için
    cv2.rectangle(img,point,(point[0]+w,point[1]+h),(0,255,0),3)

cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

fotoğraf transferi

In [ ]:
import cv2

def nothing(x):
    pass

img1 = cv2.imread("aircraft.jpg")
img1 = cv2.resize(img1,(640,480))

img2  = cv2.imread("balls.jpg")
img2 = cv2.resize(img2,(640,480))

output = cv2.addWeighted(img1,0.5,img2,0.5,0) #ilk resim, onun ağırlığı,2.resim,onun ağırlığı ,gama değeri varsayılan 0 (ağırlık toplamı 1 olmalı)

windowName = "Transition Program"
cv2.namedWindow(windowName)

cv2.createTrackbar("Alpha-Beta",windowName,0,1000,nothing)

while True:
    cv2.imshow(windowName,output)
    alpha = cv2.getTrackbarPos("Alpha-Beta",windowName)/1000
    beta = 1-alpha
    
    output = cv2.addWeighted(img1,alpha,img2,beta,0)
    print(alpha,beta)
    
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cv2.destroyAllWindows()

resimdeki metni okuma

In [ ]:
from PIL import Image
import pytesseract

img = Image.open(r"C:\Users\zeyne\Desktop\opencvkurs\temel_islemler\text.png")
text = pytesseract.image_to_string(img,lang="eng")
print(text)

araç plakası bulma

In [ ]:
import cv2
import numpy as np
import pytesseract
import imutils #opencv gibi bir kütüphane

img =cv2.imread("licence_plate.jpg")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
filtred = cv2.bilateralFilter(gray,6,250,250) #kenarları koruyarak görüntüyü yumuşattık/yarıçap,sigma renk uzayı filtreleme,space koordinat filtreleme
edged = cv2.Canny(filtred,30,200) #keskin kenarları tespit ediyoruz,min eşik,max eşik sınırı

contours = cv2.findContours(edged,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #
cnts = imutils.grab_contours(contours) 
cnts = sorted(cnts,key=cv2.contourArea,reverse=True)[:10] #key neye göre sıralayacağımız/reverse ters çevir true sırala/0 dan 10 kadar değerler için
screen = None #bulup bulmadığımzı kontrol

for c in cnts:
    epsilon = 0.018*cv2.arcLength(c,True) # epsilon doğruluk sayısı/ 0.018 deneysel sayı/arc konturların yay uzunluğunu bulma/true boşluk yoksa go
    approx = cv2.approxPolyDP(c,epsilon,True) #konturlara daha yaklaşmak için
    
    if len(approx) == 4: #4 taneyse
        screen = approx
        break
    
mask = np.zeros(gray.shape,np.uint8) # 0lardan değer oluşturuyoruz grayın içinde tuttuğu boyutlar kadar/ npzeros siyah ekran oluşturduk
new_img = cv2.drawContours(mask,[screen],0,(255,255,255),-1) #maskta screen korrdinatları aldık çizim moodu0
new_img = cv2.bitwise_and(img,img,mask=mask) # mask alanına img yapıştırıyoruz


(x,y) = np.where(mask == 255) #  beyaz bölgeyi x,y içerisinde tut
(topx,topy) = (np.min(x),np.min(y))
(botx,boty) = (np.max(x),np.max(y)) 
cropped = gray[topx:botx+1,topy:boty+1] #kırpma işlemi +1 en son değer gelsin

text = pytesseract.image_to_string(cropped,lang="eng")
print("plaka",text)

araç sayma

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(r"C:\Users\zeyne\Desktop\opencvkurs\temel_islemler\traffic.avi")
backsub = cv2.createBackgroundSubtractorMOG2()
c = 0

while True:
    ret,frame = cap.read()
    if ret: # 0 1 rette tuttuluyor true ise al diyoruz
        fgmask = backsub.apply(frame) #arka plan çıkıyoruz ön plan kalıyor
        cv2.line(frame,(50,0),(50,300),(0,255,0),2)
        cv2.line(frame,(70,0),(70,300),(0,255,0),2) #belli koordinatlar video için bulundu çizgi çekiyoruz 2 tane
        
        contours,hierarchy = cv2.findContours(fgmask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #hierarchy rsim bilgileri tutuyor hata yok ediyoz
        try: hierarchy = hierarchy[0] #hata gideriyoruz
        except: hierarchy = []
        
        for contour,hier in zip(contours,hierarchy): #çok değer var zip
            (x,y,w,h) =cv2.boundingRect(contour) #parametre çekiyoruz 
            if w> 40 and h> 40:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
                if x>50 and x<70:
                    c+= 1 #sayaç arttıyoruz
                    
        cv2.putText(frame,"car: "+str(c),(90,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),2,cv2.LINE_AA) #str aracı ekliyor/yazının bulunacağı koordinat/ FONT BÜYÜKLÜĞÜ İLK 2/AA GÜZEL GÖRÜNSÜN DİYE
        
        cv2.imshow("Car Counting",frame)
        cv2.imshow("fgmask",fgmask)
        
        if cv2.waitKey(40) & 0xFF==ord('q'):
            break

cap.release()
cv2.destroyAllWindows() 

el hareketleri algılama

In [ ]:
import cv2
import numpy as np
import math

cap = cv2.VideoCapture(0)

while (1):
    try: #hata vermemek için try except kullanıyoruz
        _,frame = cap.read()
        frame = cv2.flip(frame,1)
        kernel = np.ones((3,3),np.uint8)
        
        roi =frame[100:300,100:300]
        cv2.rectangle(frame,(100,100),(300,300),(0,0,255),0)
        
        hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
        
        lower_skin = np.array([0,20,70],np.uint8)
        upper_skin = np.array([20,255,255],np.uint8) #deri rengi renkleri netten bakılacak
        
        mask = cv2.inRange(hsv,lower_skin,upper_skin)
        mask = cv2.dilate(mask,kernel,iterations=4) #siyah noktaları beyaztlatmak için 
        mask = cv2.GaussianBlur(mask,(5,5),100) #kernel değeri 5,5 / varsayılan 100
        
        contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        cnt = max(contours,key= lambda x: cv2.contourArea(x)) #girdilerden en yükseği çıktı döndürüyor/ alanı döndürüyor sabit
        
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True) #konturlara daha iyi yaklaşım deneysel şeyler
        
        hull = cv2.convexHull(cnt) #koordinatları tutuyoruz yukardan aldığımız
        
        areaHull = cv2.contourArea(hull) #alan 
        areaCnt = cv2.contourArea(cnt) #elimizin alan hesabı
        
        areaRatio = ((areaHull-areaCnt)/areaCnt)*100 #el dışı alan
        
        hull = cv2.convexHull(approx,returnPoints=False) #true da konturlar, false indisler/ yeni alan
        defects = cv2.convexityDefects(approx,hull) #ksuur buluyoz
        
        l = 0 # kusur sayısı
        
        for i in range(defects.shape[0]): #0dan değere kadar dolandırıcak
            s,e,f,d = defects[i,0] #sırayla atıyor i +1 +1 
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0]) #satır değeri
            
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2) #uzunluk hesabı
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            d=(2*ar)/a
            
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57 #sbt
            
            if angle <= 90 and d>30:
                l+=1
                cv2.circle(roi,far,3,(255,0,0),-1)
                
            cv2.line(roi,start,end,(0,255,0),2)
            
        l+=1
        
        font =cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areaCnt<2000:
                cv2.putText(frame,'Put your hand in the box',(0,50), font, 1, (0,0,255), 3, cv2.LINE_AA)
            else:
                if areaRatio<12:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif areaRatio<17.5:
                    cv2.putText(frame,'Best Luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                   
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        
        elif l ==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l ==3:
            if areaRatio<27:
                cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                cv2.putText(frame,'OK',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
           
        elif l ==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
          
        elif l ==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        
        elif l ==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
           
        else:
            cv2.putText(frame,'',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
           
        cv2.imshow("mask",mask)
        cv2.imshow("frame",frame)

    except:
        pass
    
    k = cv2.waitKey(5) & 0xFF #esc basınca çıkacak
    if k ==27:
        break

cap.release()
cv2.destroyAllWindows()